<a href="https://colab.research.google.com/github/hBar2013/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/Kim_Lowry_LS_DS_141_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import seaborn as sns
import numpy as np

In [0]:
congress_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
congress_df = pd.read_csv(congress_url)

In [75]:
congress_df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_df.rename(columns={congress_df.columns[0]: 'party',
                    congress_df.columns[1]: 'hc_infant',
                    congress_df.columns[2]: 'water_share',
                    congress_df.columns[3]: 'budget',
                    congress_df.columns[4]: 'doc_fee',
                    congress_df.columns[5]: 'el_salvadore',
                    congress_df.columns[6]: 'school_rel',
                    congress_df.columns[7]: 'antisat',
                    congress_df.columns[8]: 'contras',
                    congress_df.columns[9]: 'missles',
                    congress_df.columns[10]: 'immigration',
                    congress_df.columns[11]: 'synfuels',
                    congress_df.columns[12]: 'ed_spend',
                    congress_df.columns[13]: 'superfund_sue',
                    congress_df.columns[14]: 'crime',
                    congress_df.columns[15]: 'duty_free',
                    congress_df.columns[16]: 'sa_export'},
                    inplace=True)

In [77]:
congress_df.head()

,party,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_df.set_index('party', inplace=True)

In [79]:
congress_df.head()

,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
party,,,,,,,,,,,,,,,,
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_nans = congress_df.replace('?', np.nan)

In [81]:
congress_nans.head()

,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
party,,,,,,,,,,,,,,,,
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
congress_num = congress_nans.replace({'y' : 1, 'n' : 0})

In [83]:
congress_num.head()

,hc_infant,water_share,budget,doc_fee,el_salvadore,school_rel,antisat,contras,missles,immigration,synfuels,ed_spend,superfund_sue,crime,duty_free,sa_export
party,,,,,,,,,,,,,,,,
republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [0]:
hc_infant = congress_num['hc_infant'].dropna()

In [85]:
hc_infant.describe()

count    422.000000
mean       0.443128
std        0.497345
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: hc_infant, dtype: float64

In [0]:
infant_r = hc_infant.loc['republican']

In [88]:
infant_r.describe()

count    164.000000
mean       0.189024
std        0.392727
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: hc_infant, dtype: float64

In [0]:
infant_d = hc_infant.loc['democrat']

In [92]:
infant_d.describe()

count    258.000000
mean       0.604651
std        0.489876
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: hc_infant, dtype: float64

In [0]:
statistic, pvalue = ttest_ind(infant_r, infant_d)

In [91]:
statistic

-9.15392122841775

In [95]:
pvalue

2.4195550274147043e-18

In [0]:
t_dist = np.random.standard_t(df=420)

In [94]:
pvalue <= 0.05

True